# LightGBM + Dask

<table>
    <tr>
        <td>
            <img src="./_img/lightgbm.svg" width="300">
        </td>
        <td>
            <img src="./_img/dask-horizontal.svg" width="300">
        </td>
    </tr>
</table>

This notebook shows how to use `lightgbm.dask` to train a LightGBM model on data stored as a [Dask Array](https://docs.dask.org/en/latest/array.html).

To explore other topics in greater depth, see the other notebooks.

<hr>

## Set up a local Dask cluster

Create a cluster with 3 workers. Since this is a `LocalCluster`, those workers are just 3 local processes.

In [1]:
from dask.distributed import Client, LocalCluster

n_workers = 3
cluster = LocalCluster(n_workers=n_workers)

client = Client(cluster)
client.wait_for_workers(n_workers)

print(f"View the dashboard: {cluster.dashboard_link}")

View the dashboard: http://127.0.0.1:8787/status


Click the link above to view a diagnostic dashboard while you run the training code below.

<hr>

## Get some training data

This example uses `sklearn.datasets.make_regression()` to generate a dataset in `numpy` format, then uses `dask.Array.from_array()` to turn that into a Dask Array.

That's just done for convenience. `lightgbm.dask` just expects that your data are Dask Arrays or Dask DataFrames.

In [2]:
import dask.array as da
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=10000, random_state=42)
dX = da.from_array(X, chunks=(1000, X.shape[1]))
dy = da.from_array(y, chunks=1000)

Right now, the Dask Arrays `data` and `labels` are lazy. Before training, you can force the cluster to compute them by running `.persist()` and then wait for that computation to finish by `wait()`-ing on them.

Doing this is optional, but it will make data loading a one-time cost so subsequent runs are fast.

In [3]:
from dask.distributed import wait

dX = dX.persist()
dy = dy.persist()
_ = wait([dX, dy])

<hr>

## Train a model

With the data set up on the workers, train a model. `lightgbm.dask.DaskLGBMRegressor` has an interface that tries to stay as close as possible to the non-Dask scikit-learn interface to LightGBM (`lightgbm.sklearn.LGBMRegressor`).

In [4]:
from lightgbm.dask import DaskLGBMRegressor

dask_reg = DaskLGBMRegressor(
    client=client,
    max_depth=5,
    objective="regression_l1",
    learning_rate=0.1,
    tree_learner="data",
    n_estimators=100,
    min_child_samples=1,
)

dask_reg.fit(
    X=dX,
    y=dy,
)

/usr/local/lib/python3.9/site-packages/lightgbm/dask.py:530: UserWarning: Parameter n_jobs will be ignored.
  _log_warning(f"Parameter {param_alias} will be ignored.")


Finding random open ports for workers
[LightGBM] [Info] Trying to bind port 37155...
[LightGBM] [Info] Binding port 37155 succeeded
[LightGBM] [Warning] Connecting to rank 2 failed, waiting for 200 milliseconds
[LightGBM] [Info] Listening...
[LightGBM] [Warning] Connecting to rank 2 failed, waiting for 260 milliseconds
[LightGBM] [Warning] Connecting to rank 2 failed, waiting for 338 milliseconds
[LightGBM] [Warning] Connecting to rank 2 failed, waiting for 439 milliseconds
[LightGBM] [Warning] Connecting to rank 2 failed, waiting for 570 milliseconds
[LightGBM] [Info] Trying to bind port 39687...
[LightGBM] [Info] Binding port 39687 succeeded
[LightGBM] [Warning] Connecting to rank 2 failed, waiting for 200 milliseconds
[LightGBM] [Info] Listening...
[LightGBM] [Info] Trying to bind port 58167...
[LightGBM] [Info] Binding port 58167 succeeded
[LightGBM] [Info] Listening...
[LightGBM] [Info] Connected to rank 1
[LightGBM] [Info] Connected to rank 2
[LightGBM] [Info] Connected to rank 0

DaskLGBMRegressor(client=<Client: 'tcp://127.0.0.1:39223' processes=3 threads=3, memory=6.79 GiB>,
                  max_depth=5, min_child_samples=1, num_threads=1,
                  objective='regression_l1', time_out=120, tree_learner='data')

<hr>

## Evaluate the model

The `.predict()` method takes in a Dask collection and returns a Dask Array.

In [5]:
preds = dask_reg.predict(dX)
print(str(preds))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
dask.array<_predict_part, shape=(10000,), dtype=float32, chunksize=(1000,), chunktype=numpy.ndarray>


Before calculating the mean absolute error (MAE) of these predictions, compute some summary statistics on the target variable. This is necessary to understand what "good" values of MAE look like.

In [6]:
p = [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]
dy_percentiles = da.percentile(dy, p).compute()

for i, percentile in enumerate(p):
    print(f"{percentile * 100}%: {round(dy_percentiles[i], 2)}")

1.0%: -627.63
10.0%: -566.9
25.0%: -515.82
50.0%: -468.06
75.0%: -452.54
90.0%: -435.72
99.0%: -419.1


The metrics functions from `dask-ml` match those from `scikit-learn`, but take in and return Dask collections. You can use these functions to perform model evaluation without the evaluation data or predictions needing to be pulled down to the machine running this notebook. Pretty cool, right?

In [7]:
from dask_ml.metrics.regression import mean_absolute_error

mean_absolute_error(preds, dy)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^m

23.59181626875831

<hr>

## Next Steps

Learn more: https://lightgbm.readthedocs.io/en/latest/Python-API.html#dask-api.

Ask a question, report a bug, or submit a feature request: https://github.com/microsoft/LightGBM/issues.

Contribute: https://github.com/microsoft/LightGBM/issues?q=is%3Aissue+is%3Aopen+label%3Adask.